<a href="https://colab.research.google.com/github/adcaceres69/DISTRIBUCION/blob/main/v2_DISTRIBUCION_Y_ROTACION_PERSONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Plan de Trabajo
1. Creación de Personas Ficticias: Generaremos 90 personas ficticias, con los roles y géneros especificados.
2. Formación de Grupos para Puestos de Control: Asignaremos estas personas a grupos específicos para cada puesto de control, asegurándonos de cumplir con los requisitos de supervisores y fiscalizadores, y la condición especial de Perin.
3. Planificación de Turnos y Descansos: Organizaremos los turnos de trabajo y los periodos de descanso para cada grupo según las modalidades especificadas.
4. Rotación de Grupos entre Guardias Cortas y Largas: Finalmente, planificaremos la rotación de los grupos entre guardias cortas (Mansilla y Colorado) y largas (otros puestos), en Ciclos de un mes.

In [1]:
# librerias
import pandas as pd
import numpy as np
import random

from collections import namedtuple, Counter
from datetime import timedelta, datetime

In [ ]:
# Punto 1: Creación de Personas Ficticias
total_varones = 51
total_mujeres = 39
supervisores_varones = 12
supervisores_mujeres = 16
identificaciones = [f"ID_{i+1}" for i in range(total_varones + total_mujeres)]

# Reajustando la asignación para el puesto "Perin" y equilibrando la proporción de géneros
# Reorganizando la lista de personas
personas = []
for i in range(total_varones):
    rol = 'supervisor' if i < supervisores_varones else 'fiscalizador'
    personas.append({'id': identificaciones[i], 'genero': 'varón', 'rol': rol})

for i in range(total_mujeres):
    rol = 'supervisor' if i < supervisores_mujeres else 'fiscalizador'
    personas.append({'id': identificaciones[i + total_varones], 'genero': 'mujer', 'rol': rol})

random.shuffle(personas)

# Punto 2: Formación de Grupos para Puestos de Control
grupos = {
    'Mansilla': {'cantidad': 4, 'por_grupo': 8, 'supervisores': 2, 'fiscalizadores': 6},
    'Colorado': {'cantidad': 3, 'por_grupo': 7, 'supervisores': 2, 'fiscalizadores': 5},
    'Perin': {'cantidad': 2, 'por_grupo': 3, 'supervisores': 1, 'fiscalizadores': 2},  # Solo varones
    'Fermin': {'cantidad': 2, 'por_grupo': 4, 'supervisores': 1, 'fiscalizadores': 3},
    'Ruta 86': {'cantidad': 2, 'por_grupo': 4, 'supervisores': 1, 'fiscalizadores': 3},
    'Barilari': {'cantidad': 2, 'por_grupo': 4, 'supervisores': 1, 'fiscalizadores': 3}
}
# Reset de las asignaciones y conteos
asignacion_grupos = {puesto: [] for puesto in grupos}
supervisores_restantes = {'varón': supervisores_varones, 'mujer': supervisores_mujeres}
fiscalizadores_restantes = {'varón': total_varones - supervisores_varones, 'mujer': total_mujeres - supervisores_mujeres}

# Asignación equilibrada de personal a grupos, asegurando que Perin tenga solo varones
for puesto, info in grupos.items():
    for _ in range(info['cantidad']):
        grupo = []
        # Asignando supervisores
        for _ in range(info['supervisores']):
            for persona in personas:
                if persona['rol'] == 'supervisor' and supervisores_restantes[persona['genero']] > 0 and (puesto != 'Perin' or persona['genero'] == 'varón'):
                    grupo.append(persona['id'])
                    supervisores_restantes[persona['genero']] -= 1
                    personas.remove(persona)
                    break
        # Asignando fiscalizadores
        for _ in range(info['fiscalizadores']):
            for persona in personas:
                if persona['rol'] == 'fiscalizador' and fiscalizadores_restantes[persona['genero']] > 0 and (puesto != 'Perin' or persona['genero'] == 'varón'):
                    grupo.append(persona['id'])
                    fiscalizadores_restantes[persona['genero']] -= 1
                    personas.remove(persona)
                    break
        asignacion_grupos[puesto].append(grupo)

# Muestra una parte de los resultados para verificar
ejemplo_asignacion_grupos = {puesto: asignacion_grupos[puesto][:1] for puesto in asignacion_grupos}
ejemplo_asignacion_grupos

In [ ]:
# Punto 3: Planificación de Turnos y Descansos

# Configuración de los turnos y descansos para cada puesto
configuracion_turnos = {
    'Mansilla': {'turno': 12, 'descanso': 36},  # horas
    'Colorado': {'turno': 8, 'descanso': 72},  # horas
    'Perin': {'turno': 7, 'descanso': 7},      # días
    'Fermin': {'turno': 7, 'descanso': 7},     # días
    'Ruta 86': {'turno': 7, 'descanso': 7},    # días
    'Barilari': {'turno': 7, 'descanso': 7}    # días
}

# Función para calcular los turnos y descansos
def calcular_turnos(puesto, inicio_mes):
    turnos = []
    fecha_actual = inicio_mes
    turno_horas = configuracion_turnos[puesto]['turno']
    descanso_horas = configuracion_turnos[puesto]['descanso']
    while fecha_actual.month == inicio_mes.month:
        fin_turno = fecha_actual + timedelta(hours=turno_horas) if puesto in ['Mansilla', 'Colorado'] else fecha_actual + timedelta(days=turno_horas)
        turnos.append(('Turno', fecha_actual, fin_turno))
        fecha_actual = fin_turno + timedelta(hours=descanso_horas) if puesto in ['Mansilla', 'Colorado'] else fin_turno + timedelta(days=descanso_horas)
    return turnos

# Inicio del mes para calcular los turnos (Enero 2024)
inicio_mes = datetime(2024, 1, 1)

# Calculando turnos y descansos para cada puesto
turnos_puestos = {puesto: calcular_turnos(puesto, inicio_mes) for puesto in configuracion_turnos}

# Muestra una parte de los resultados para verificar
ejemplo_turnos_puestos = {puesto: turnos_puestos[puesto][:2] for puesto in turnos_puestos}  # Primeros dos turnos de cada puesto
ejemplo_turnos_puestos


In [ ]:
# Punto 4: Rotación de Grupos entre Guardias Cortas y Largas en Ciclos de un Mes

# Definición de guardias cortas y largas
guardias_cortas = ['Mansilla', 'Colorado']
guardias_largas = ['Perin', 'Fermin', 'Ruta 86', 'Barilari']

# Asignación inicial de grupos a guardias cortas y largas
grupos_guardias_cortas = [grupo for puesto, grupos in asignacion_grupos.items() if puesto in guardias_cortas for grupo in grupos]
grupos_guardias_largas = [grupo for puesto, grupos in asignacion_grupos.items() if puesto in guardias_largas for grupo in grupos]

# Plan de rotación para los próximos meses
plan_rotacion = {}
total_meses = 6  # Ejemplo para 6 meses

# Rotación mensual
for mes in range(1, total_meses + 1):
    plan_rotacion[mes] = {
        'guardias_cortas': grupos_guardias_cortas.copy(),
        'guardias_largas': grupos_guardias_largas.copy()
    }
    # Intercambio de grupos entre guardias cortas y largas
    grupos_guardias_cortas, grupos_guardias_largas = grupos_guardias_largas, grupos_guardias_cortas

# Muestra una parte de los resultados para verificar
ejemplo_plan_rotacion = {mes: plan_rotacion[mes] for mes in range(1, 4)}
ejemplo_plan_rotacion


In [5]:
%whos

Variable                    Type        Data/Info
-------------------------------------------------
Counter                     type        <class 'collections.Counter'>
asignacion_grupos           dict        n=6
calcular_turnos             function    <function calcular_turnos at 0x7bca9e2ee5f0>
configuracion_turnos        dict        n=6
datetime                    type        <class 'datetime.datetime'>
ejemplo_asignacion_grupos   dict        n=6
ejemplo_plan_rotacion       dict        n=3
ejemplo_turnos_puestos      dict        n=6
fiscalizadores_restantes    dict        n=2
grupo                       list        n=4
grupos                      dict        n=6
grupos_guardias_cortas      list        n=7
grupos_guardias_largas      list        n=8
guardias_cortas             list        n=2
guardias_largas             list        n=4
i                           int         38
identificaciones            list        n=90
info                        dict        n=4
inicio_mes       